In [ ]:
!gdown  --id 1g0Y27WxxsQCWWagNqUvajlre3tDOKsbM

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1g0Y27WxxsQCWWagNqUvajlre3tDOKsbM
To: /content/syntactic_car_factory_data.csv
100% 528k/528k [00:00<00:00, 46.5MB/s]


In [ ]:
import pandas as pd
data = pd.read_csv("/content/syntactic_car_factory_data.csv")
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     3000 non-null   int64  
 1   Sensor_1       2471 non-null   float64
 2   Sensor_2       2439 non-null   float64
 3   Sensor_3       2460 non-null   float64
 4   Sensor_4       2422 non-null   float64
 5   Sensor_5       2439 non-null   float64
 6   Sensor_6       2477 non-null   float64
 7   Sensor_7       2467 non-null   float64
 8   Sensor_8       2436 non-null   float64
 9   Sensor_9       2415 non-null   float64
 10  Sensor_10      2478 non-null   float64
 11  Sensor_Status  2469 non-null   object 
 12  Shift          2451 non-null   object 
 13  Line           2470 non-null   object 
dtypes: float64(10), int64(1), object(3)
memory usage: 328.2+ KB


In [ ]:
data['Sensor_1'] = data['Sensor_1'].fillna(data['Sensor_1'].mean())
data['Shift'] = data['Shift'].fillna(data['Shift'].mode()[0])
data['Line'] = data['Line'].fillna(data['Line'].mode()[0])


## Find the relationship between sensor 1 readings and shifts

In [ ]:
from scipy.stats import f_oneway
# Group sensor readings by shifts
sensor_readings = [
    data[data['Shift'] == shift]['Sensor_1'] for
    shift in data['Shift'].unique()
]

anova_result = f_oneway(*sensor_readings)
print(anova_result)
if anova_result.pvalue < 0.05:
    print("There is a significant difference in sensor readings between shifts.")
else:
    print("There is no significant difference in sensor readings between shifts.")

F_onewayResult(statistic=0.5494456374551399, pvalue=0.5773278760339522)
There is no significant difference in sensor readings between shifts.


In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

# independent variable shit and line
# dependent variable Sensor_1
model = ols('Sensor_1 ~ C(Shift) + C(Line) + C(Shift):C(Line)',
            data=data).fit()
anova_results  = sm.stats.anova_lm(model, type = 2)
print(anova_results)
# Interpretation
alpha = 0.05



                      df         sum_sq    mean_sq         F    PR(>F)
C(Shift)             2.0      95.267185  47.633592  0.548691  0.577764
C(Line)              2.0      28.947839  14.473920  0.166725  0.846440
C(Shift):C(Line)     4.0     134.482480  33.620620  0.387276  0.817889
Residual          2991.0  259658.227414  86.813182       NaN       NaN
